In [1]:
!pip install transformers
!pip install datasets
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 62.1 MB/s 
     |████████████████████████████████| 163 kB 78.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 68.9 MB/s 
     |████████████████████████████████| 115 kB 72.1 MB/s 
     |████████████████████████████████| 127 kB 86.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=84b34253ab12a615a0c67b7cbb29069110ac9ed85c2aa

In [1]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration
from datasets import Dataset
import tensorflow as tf
from tensorflow.keras import optimizers
from rouge_score import rouge_scorer
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [3]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog.Hey whats up . this is cool . You hate me ?',
                      'The quick brown dog jumps on the log. You hate me ?')
k=scores['rouge1']
k[1]

0.5

In [4]:
articles = []
summaries = []

In [5]:
for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Articles/business'):
  article = open(os.path.join('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Articles/business',f),'r')
  next(article)
  article = article.read()
  article = article.replace('\n','')
  article = article.replace('\\','')
  articles.append(article)

for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Articles/entertainment'):
  article = open(os.path.join('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Articles/entertainment',f),'r')
  next(article)
  article = article.read()
  article = article.replace('\n','')
  article = article.replace('\\','')
  articles.append(article)

In [6]:
for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Summaries/business'):
  Summary = open(os.path.join('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Summaries/business',f),'r')

  Summary = Summary.read()
  Summary = Summary.replace('\n','')
  Summary = Summary.replace('\\','')
  summaries.append(Summary)

for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Summaries/entertainment'):
  Summary = open(os.path.join('/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/Datasets/Summaries/entertainment',f),'r')

  Summary = Summary.read()
  Summary = Summary.replace('\n','')
  Summary = Summary.replace('\\','')
  summaries.append(Summary)

In [3]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Abstractive Text Summarizer using encoder decoder model/news_summary_more.csv")

In [7]:
x_train,x_test,y_train,y_test = train_test_split(data['text'][:2000],data['headlines'][:2000],test_size=0.2,random_state=42)

In [8]:
def tokenize_convert_tfdataset(tokenizer,x, y):
  prefix = "summarize: "
  x = [prefix+article for article in x]
  data_dict = {'article' : x , 'summary' : y }
  data = Dataset.from_dict(data_dict)
  input = data.map(lambda e: tokenizer( e['article'], truncation=True, padding='max_length',max_length=512,return_tensors='tf'), batched=True)
  output = data.map(lambda e: tokenizer(text_target =  e['summary'], truncation=True, padding='max_length',max_length=100,return_tensors='tf'), batched=True)
  input.set_format(type='tensorflow', columns=['input_ids','attention_mask'])
  output.set_format(type='tensorflow', columns=['input_ids'])

  input_ids = input['input_ids']
  attention_mask = input['attention_mask']
  labels = output['input_ids']
  labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels ]

  features_dict = {
      'input_ids' : input_ids,
      'attention_mask' : attention_mask,
      'labels' : labels
  }
  tfdataset = tf.data.Dataset.from_tensor_slices(features_dict).shuffle(1000).batch(3)
  return tfdataset

In [9]:
train_dataset = tokenize_convert_tfdataset(tokenizer, x_train,y_train)
valid_dataset = tokenize_convert_tfdataset(tokenizer, x_test,y_test)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
print(tokenizer.pad_token_id)

0


In [13]:
def Rouge_S(y_true,y_pred):
  k = tf.cast(y_true,tf.int32).numpy()
  h = tf.cast(y_pred,tf.int32).numpy()
  k = np.where(k != -100, k, tokenizer.pad_token_id)
  print(h)
  ytrue_tok = tokenizer.batch_decode(k, skip_special_tokens=True)
  ypred_tok = tokenizer.batch_decode(h, skip_special_tokens=True)
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  scores = scorer.score(ytrue_tok,ypred_tok)
  return scores

In [10]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
model.compile(optimizer=optimizers.Adam(learning_rate=4e-5))
output = model.fit(train_dataset,
          epochs=3,
          batch_size=3,
          validation_data=valid_dataset)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/3
534/534 [==============================] - 388s 665ms/step - loss: 1.7060 - val_loss: 1.3088
Epoch 2/3
534/534 [==============================] - 347s 651ms/step - loss: 1.2636 - val_loss: 1.2702
Epoch 3/3
534/534 [==============================] - 348s 652ms/step - loss: 1.0502 - val_loss: 1.2646


In [12]:
text = data['text'][6000]
input_ids = tokenizer("summarize: " + text, return_tensors="tf").input_ids
outputs = model.generate(input_ids,max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Delhi air quality has improved by 10% to 15% in past 2 years


In [18]:
print(data['headlines'][6000])


Delhi air quality improved 10% in past couple of years


In [29]:
print(articles[0])

Billions of farmers' livelihoods are at risk from falling commodity prices and protectionism, the UN's Food & Agriculture Organisation has warned.Trade barriers and subsidies "severely" distort the market, the FAO report on the "State of Agricultural Commodity Markets 2004" said. As a result, the 2.5 billion people in the developing world who rely on farming face food insecurity. The most endangered are those who live in the least-developed countries. The FAO report said that support for farmers in industrialised nations was equivalent to 30 times the amount provided as aid for agricultural development in poor countries.The FAO has urged the World Trade Organisation to swiftly conclude negotiations to liberalise trade, easing developing countries' access to the world market. It also criticised the high tariffs imposed by both developed and developing nations. It recommends that developing countries reduce their own tariffs to encourage trade and take advantage of market liberalisation.